In [1]:
import os

BEARER_TOKEN = os.environ.get("BEARER_TOKEN") or "BEARER_TOKEN_HERE"
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}"
}

In [17]:
import requests
from requests.adapters import HTTPAdapter, Retry
batch_size = 100
endpoint_url = "http://localhost:8000"
s = requests.Session()

# we setup a retry strategy to retry on 5xx errors
retries = Retry(
    total=5,  # number of retries before raising error
    backoff_factor=0.1,
    status_forcelist=[500, 502, 503, 504]
)
s.mount('http://', HTTPAdapter(max_retries=retries))

In [13]:
import requests
endpoint_url = "http://localhost:8000"

query = "What is the equity analysts' view on NEL ASA"
res = s.post(
        f"{endpoint_url}/query",
        headers=headers,
        json={'query': query}
    )


In [1]:
from app.src.chatgpt import ChatGPT
import os
chat = ChatGPT(os.environ['OPENAI_API_KEY'], 
                                 os.environ['PINECONE_API_KEY'], 
                                 os.environ['PINECONE_ENVIRONMENT'],
                                 os.environ['PINECONE_INDEX']
                                 )

/Users/frances/.local/share/virtualenvs/finance-qa-zx6p6h4E/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
from src.models.agents import Agent
from src.models.models import Query, Type, PromptType
agent = Agent()

In [8]:
query = "CEVA Logistics partners with Toyota Motor Europe to test new Hydrogen Fuel Cell Truck?"
#filter = {"type": Type.table}
q = Query(query=query, top_k=10)

In [19]:
import openai
import os

In [20]:
openai.key = os.environ["OPENAI_API_KEY"]

In [30]:
messages=[
        {'role': 'user', 'content': 'Count to 10, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ]

In [34]:
import time
start_time = time.time()
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = messages,
    temperature = 0,
    stream=True
)

final_text = ''
for chunk in response:
    if chunk['choices'][0]['finish_reason'] == 'stop':
        break
    text = chunk['choices'][0]['delta']['content']
    final_text += text
    print(f"{text}", end='', flush=True)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10

In [42]:
def submit_query():
    messages = [
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ]

    for response in openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=0,
        stream=True
    ):
        yield response['choices'][0]['delta']['content']

In [46]:
def submit_query():
    messages = [
        {'role': 'user', 'content': 'Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ]

    for response in openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=0,
        stream=True
    ):
        if response['choices'][0]['finish_reason'] == 'stop':
            break
        yield response['choices'][0]['delta']['content']


def update_streaming_text():
    for text in submit_query():
        print(text, end='', flush=True)

In [47]:
update_streaming_text()